# 🫁 Multimodal Survival Modeling in Pulmonary Tuberculosis
#### Chest X-ray + Clinical Covariates via Cross-Modal Attention

**Author:** Dr. Ikechukwu Ephraim Ugbo, MD
**Project:** Innovative AI Healthcare Solutions
**Framework:** TensorFlow / Keras

#### ---
#### 🔬 Study Context & SOTA References
##### This notebook implements a state-of-the-art (SOTA) multimodal architecture for predicting **time-to-major-complication** in TB patients.

##### Moving beyond simple feature concatenation, we implement **Cross-Modal Attention** to allow clinical covariates (e.g., HIV status, Age) to dynamically "attend" to specific regions of the chest X-ray. This approach is supported by recent literature:

##### **Architecture:** We utilize a transformer-based fusion strategy, which has shown superior performance in handling heterogeneous medical data compared to unimodal baselines.
##### **Modality Interaction:** Modeling the complex non-linear interactions between imaging and clinical variables (e.g., "how does diabetes status alter the risk implication of a cavitary lesion?") is critical for robust prognosis.
##### **Backbone:** We use a DenseNet121 backbone for visual feature extraction, a proven strategy for medical imaging tasks.
#
#### ---

In [2]:
# ==========================================
# Imports & Setup
# ==========================================
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

# Import custom modules
# Ensure your src/ folder is accessible in the python path
from src.config import *
from src.data_utils import (
    generate_synthetic_tb_clinical_data, 
    load_tb_cxr_dataset, 
    train_val_split,
    # Assuming you added this function based on our previous step:
    # create_matched_dataframe 
)
from src.model_utils import TBSurvivalNet
from src.survival_utils import harrell_c_index
from src.training_utils import compile_survival_model
from src.explainability_utils import generate_gradcam

# Set seeds for reproducibility
tf.random.set_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

print(f"TensorFlow Version: {tf.__version__}")
print(f"GPUs Available: {tf.config.list_physical_devices('GPU')}")

ModuleNotFoundError: No module named 'pandas'

#### 1. Data Preparation & Synchronization

**Challenge:** A common issue in multimodal medical AI is data heterogeneity and missing modalities.

**Strategy:**
1. We load the raw **Shenzhen TB Chest X-ray** dataset.
2. We generate synchronized synthetic clinical data (Age, HIV, etc.) that matches the *exact* number of images.
3. We create a "Master Key" dataframe to ensure Image $i$ always corresponds to Clinical Vector $i$.

In [ ]:
# ==========================================
# Data Loading
# ==========================================
# Define path to your image directory
IMAGE_DATA_DIR = "data/raw/images/"  # Update this path if different

# 1. Generate/Load Clinical Data matched to Images
# (Logic: Find all images, generate N clinical rows, save 'master_dataset.csv')
if not os.path.exists("data/processed/master_dataset.csv"):
    print("Generating matched clinical data...")
    # NOTE: Ensure you updated src/data_utils.py with 'create_matched_dataframe'
    # If not, use the standalone logic we discussed.
    from src.data_utils import create_matched_dataframe 
    df = create_matched_dataframe(IMAGE_DATA_DIR, output_path="data/processed/master_dataset.csv")
else:
    print("Loading existing master dataset...")
    df = pd.read_csv("data/processed/master_dataset.csv")

# 2. Load Images using the fixed loader
# This fixes the NameError from the previous version
ds_images = load_tb_cxr_dataset(
    data_dir=IMAGE_DATA_DIR,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False # Important: Keep order to match tabular data first
)

# 3. Prepare Tabular Data
clinical_features = ["age", "sex", "hiv", "diabetes", "bmi", "hemoglobin", "albumin"]
X_tabular = df[clinical_features].values.astype("float32")
time_event = df[["time", "event"]].values

print(f"Clinical Data Shape: {X_tabular.shape}")
print(f"Total Images Found: {len(df)}")

#### 2. Baseline Model: Cox Proportional Hazards

Before training the Deep Learning model, we establish a clinical baseline.
This helps us quantify the "added value" of the imaging modality.

In [ ]:
# ==========================================
# Baseline Cox Proportional Hazards Model
# ==========================================
from lifelines import CoxPHFitter

# Fit Cox Model on Clinical Data Only
cph = CoxPHFitter()
cph_df = df[clinical_features + ["time", "event"]]
cph.fit(cph_df, duration_col="time", event_col="event")

# Evaluate
c_index_cox = cph.concordance_index_
print(f"Baseline Clinical-Only C-Index: {c_index_cox:.4f}")

# cph.print_summary() # Uncomment to see Hazard Ratios

#### 3. SOTA Multimodal Architecture

We implement the **Cross-Modal Attention Network** (TBSurvivalNet).

#### How it works:
1. **Image Encoder:** `DenseNet121` (without pooling) extracts a $7 \times 7$ grid of visual features.
2. **Tabular Encoder:** Clinical features are projected into a high-dimensional embedding space.
3. **Cross-Attention:** The clinical embedding acts as a "Query" to attend to the spatial visual "Keys/Values". This allows the model to focus on specific lung regions relevant to the patient's specific comorbidities.

In [ ]:
# ==========================================
# Model Initialization
# ==========================================
from tensorflow.keras.applications import DenseNet121

# 1. Define Visual Backbone
# CRITICAL CHANGE: pooling=None.
# We need the spatial (7,7,1024) features for Attention, not the global average.
image_encoder = DenseNet121(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3),
    pooling=None 
)
image_encoder.trainable = False # Freeze initially

# 2. Initialize SOTA Model
# (Ensure src/model_utils.py has been updated with the CrossModalAttention class)
model = TBSurvivalNet(
    image_encoder=image_encoder,
    tabular_dim=X_tabular.shape[1],
    embed_dim=256
)

# 3. Build & Summary
# Input 1: Images (Batch, 224, 224, 3)
# Input 2: Tabular (Batch, N_features)
model.build([
    (None, 224, 224, 3), 
    (None, X_tabular.shape[1])
])

model.summary()

In [ ]:
# ==========================================
# Training Setup
# ==========================================
# Split Data (Train/Val)
# Note: In a real run, you must ensure X_img corresponds to these indices.
# Here we demonstrate the compilation.

model = compile_survival_model(
    model,
    lr=1e-4 # Lower LR is often better for fine-tuning multimodal models
)

print("Model compiled with Cox Partial Likelihood Loss.")

#### 4. Training & Explainability

Post-training, we will use **Grad-CAM** to visualize which lung regions the model focused on.

> "Clinicians require transparent, actionable models for adoption in decision-making."

In [ ]:
# generate_gradcam(
#     model,
#     image=X_img_val[0],
#     tabular_dim=X_tabular.shape[1],
#     layer_name="conv5_block16_concat"
# )

In [ ]:
# ==========================================
# Placeholder Training Loop
# ==========================================
history = model.fit(
   x=[X_img_train, X_tab_train],
   y=y_train_surv, # (time, event)
   validation_data=([X_img_val, X_tab_val], y_val_surv),
   epochs=20, # Reduced for demo
   batch_size=BATCH_SIZE
)

# Save weights for future inference
# model.save_weights("outputs/models/sota_tb_survival_v1.h5")
print("Ready for training loop execution.")

## Interpretation and Next Steps

- Compare clinical vs multimodal performance
- Perform subgroup analyses (HIV, age)
- External validation
- Competing risks modeling
- Manuscript preparation


### Summary

This notebook provides a reproducible, interpretable framework
for survival modeling in pulmonary tuberculosis using
chest X-ray imaging and clinical data.

It serves as the foundation for further validation and publication
within the **Innovative AI Healthcare Solutions** initiative.
